# Análisis de Sentimientos IMDB

Los datos están divididos equitativamente con 25k reseñas destinadas para entrenamiento y 25k para probar tu clasificador. Además, cada conjunto tiene 12.5k reseñas positivas y 12.5k negativas.

IMDb permite a los usuarios calificar películas en una escala del 1 al 10. Para etiquetar estas reseñas, el curador de los datos etiquetó cualquier cosa con ≤ 4 estrellas como negativa y cualquier cosa con ≥ 7 estrellas como positiva. Las reseñas con 5 o 6 estrellas fueron excluidas.

**Importar las librerías necesarias**

In [29]:
# En Google Colab !!!

In [1]:
# =============================================================================
# IMPORTACIÓN DE LIBRERÍAS NECESARIAS
# =============================================================================
# numpy: para operaciones numéricas y manejo de arrays
# pandas: para manipulación de datos estructurados (aunque no se usa mucho aquí)
# os: para operaciones con el sistema operativo (rutas de archivos)
# re: para expresiones regulares (limpieza de texto)
# warnings: para suprimir mensajes de advertencia que no son críticos

import numpy as np
import pandas as pd
import os
import re
import warnings
warnings.filterwarnings("ignore")  # Ignoramos warnings para tener output más limpio

**Cargar Datos**

In [31]:
#reviews_train = []
#for line in open(os.getcwd() + '/data/imbd_train.txt', 'r', encoding='latin1'):

#    reviews_train.append(line.strip())

#reviews_test = []
#for line in open(os.getcwd() + '/data/imbd_test.txt', 'r', encoding='latin1'):

#    reviews_test.append(line.strip())

In [2]:
# =============================================================================
# CARGA DE DATOS - REVIEWS DE PELÍCULAS
# =============================================================================
# Cargamos las reviews de entrenamiento desde el archivo de texto
# Cada línea del archivo es una review completa de una película

reviews_train = []  # Lista para almacenar las reviews de entrenamiento
for line in open('imbd_train.txt', 'r', encoding='latin1'):
    # strip() elimina espacios en blanco y saltos de línea al inicio y final
    reviews_train.append(line.strip())

# Cargamos las reviews de test (para evaluar el modelo después)
reviews_test = []  # Lista para almacenar las reviews de test
for line in open('imbd_test.txt', 'r', encoding='latin1'):
    reviews_test.append(line.strip())

# Dataset: 25,000 reviews de entrenamiento y 25,000 de test
# Train: primeras 12,500 son positivas, últimas 12,500 son negativas
# Test: misma estructura

In [3]:
# =============================================================================
# EXPLORACIÓN INICIAL - Visualizar algunas reviews
# =============================================================================
# Imprimimos las primeras 3 reviews para ver cómo se ven los datos originales
# Esto nos ayuda a identificar qué tipo de limpieza necesitaremos después

for i in range(3):
    print('####################')
    print(reviews_train[i])

####################
Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!
####################
Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once conside

**Ver uno de los elementos de la lista**

In [4]:
# =============================================================================
# VERIFICAR TAMAÑO DEL DATASET Y VER UN EJEMPLO
# =============================================================================
# Comprobamos cuántas reviews tenemos en cada conjunto

print(len(reviews_train))  # Debería ser 25,000
print(len(reviews_test))   # Debería ser 25,000

# Mostramos una review específica (la número 5) para ver su contenido
reviews_train[5]

25000
25000


"This isn't the comedic Robin Williams, nor is it the quirky/insane Robin Williams of recent thriller fame. This is a hybrid of the classic drama without over-dramatization, mixed with Robin's new love of the thriller. But this isn't a thriller, per se. This is more a mystery/suspense vehicle through which Williams attempts to locate a sick boy and his keeper.<br /><br />Also starring Sandra Oh and Rory Culkin, this Suspense Drama plays pretty much like a news report, until William's character gets close to achieving his goal.<br /><br />I must say that I was highly entertained, though this movie fails to teach, guide, inspect, or amuse. It felt more like I was watching a guy (Williams), as he was actually performing the actions, from a third person perspective. In other words, it felt real, and I was able to subscribe to the premise of the story.<br /><br />All in all, it's worth a watch, though it's definitely not Friday/Saturday night fare.<br /><br />It rates a 7.7/10 from...<br />

El texto sin procesar está bastante desordenado para estas reseñas, así que antes de poder hacer cualquier análisis necesitamos limpiar las cosas


**Usar expresiones regulares para eliminar los caracteres que no son texto y las etiquetas html**

In [5]:
# =============================================================================
# LIMPIEZA Y PREPROCESAMIENTO DEL TEXTO
# =============================================================================
# Las reviews tienen ruido: puntuación, etiquetas HTML, números, etc.
# Necesitamos limpiar el texto para que el modelo se enfoque en las palabras importantes

import re

# PASO 1: Definir patrones de expresiones regulares para limpieza

# REPLACE_NO_SPACE: elimina caracteres que NO queremos (los sustituye por cadena vacía)
# Incluye: puntos, punto y coma, dos puntos, exclamaciones, interrogaciones, comas,
#          comillas, paréntesis, corchetes y números
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")

# REPLACE_WITH_SPACE: caracteres que queremos sustituir por espacios
# Incluye: etiquetas HTML como <br/><br/>, guiones y barras
# Los sustituimos por espacio para no juntar palabras
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

# Constantes para las sustituciones
NO_SPACE = ""
SPACE = " "

# PASO 2: Función de preprocesamiento
def preprocess_reviews(reviews):
    """
    Limpia una lista de reviews de texto:
    1. Convierte todo a minúsculas (para normalizar)
    2. Elimina signos de puntuación y números
    3. Sustituye etiquetas HTML y guiones por espacios
    
    Args:
        reviews: lista de strings con las reviews originales
    
    Returns:
        reviews_clean: lista de strings con las reviews limpias
    """
    # Primera pasada: eliminar signos de puntuación y números
    # .lower() convierte todo a minúsculas
    # .sub() sustituye lo que coincida con el patrón
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    
    # Segunda pasada: sustituir etiquetas HTML y guiones por espacios
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

# PASO 3: Aplicar la limpieza a nuestros datos
reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

# Ahora las reviews están limpias y listas para vectorización

In [6]:
# =============================================================================
# COMPARAR: Review antes y después de la limpieza
# =============================================================================
# Mostramos la misma review (índice 5) después de la limpieza
# Compara con la versión original que vimos antes
# Observa: texto en minúsculas, sin puntuación, sin números, sin HTML

reviews_train_clean[5]

"this isn't the comedic robin williams nor is it the quirky insane robin williams of recent thriller fame this is a hybrid of the classic drama without over dramatization mixed with robin's new love of the thriller but this isn't a thriller per se this is more a mystery suspense vehicle through which williams attempts to locate a sick boy and his keeper also starring sandra oh and rory culkin this suspense drama plays pretty much like a news report until william's character gets close to achieving his goal i must say that i was highly entertained though this movie fails to teach guide inspect or amuse it felt more like i was watching a guy williams as he was actually performing the actions from a third person perspective in other words it felt real and i was able to subscribe to the premise of the story all in all it's worth a watch though it's definitely not friday saturday night fare it rates a   from the fiend "

# Vectorización
Para que estos datos tengan sentido para nuestro algoritmo de aprendizaje automático necesitaremos convertir cada reseña a una representación numérica, que llamamos vectorización.

La forma más simple de esto es crear una matriz muy grande con una columna para cada palabra única en tu corpus (donde el corpus son todas las 50k reseñas en nuestro caso). Luego transformamos cada reseña en una fila que contiene 0s y 1s, donde 1 significa que la palabra del corpus correspondiente a esa columna aparece en esa reseña. Dicho esto, cada fila de la matriz será muy dispersa (mayormente ceros). Este proceso también se conoce como codificación one-hot. Usar el método *CountVectorizer*.

In [7]:
# =============================================================================
# IMPORTAR CountVectorizer
# =============================================================================
# CountVectorizer es la herramienta de sklearn que convierte texto en vectores numéricos
# Es necesario porque los modelos de ML solo entienden números, no palabras

from sklearn.feature_extraction.text import CountVectorizer

In [8]:
# =============================================================================
# EJEMPLO SIMPLE - Corpus de demostración
# =============================================================================
# Creamos un corpus pequeño de 4 documentos para entender cómo funciona la vectorización
# Este ejemplo nos ayudará a visualizar el proceso antes de aplicarlo a nuestras 25,000 reviews

corpus = [
     'This is the first document.',      # Documento 0
     'This document is the second document.',  # Documento 1
     'And this is the third one.',       # Documento 2
     'Is this the first document?',      # Documento 3
]

In [9]:
# =============================================================================
# VECTORIZACIÓN - Ejemplo básico
# =============================================================================
# Proceso de vectorización:
# 1. El vectorizador identifica todas las palabras únicas del corpus (vocabulario)
# 2. Cada palabra única se convierte en una "feature" (columna)
# 3. El vectorizador crea una matriz donde cada fila es un documento

vectorizer = CountVectorizer()  # Creamos el vectorizador
X = vectorizer.fit_transform(corpus)  # fit: aprende el vocabulario, transform: convierte a matriz

# Veamos qué palabras encontró (el vocabulario, ordenado alfabéticamente)
vectorizer.get_feature_names_out()

array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third',
       'this'], dtype=object)

In [10]:
# =============================================================================
# TAMAÑO DEL VOCABULARIO
# =============================================================================
# Contamos cuántas palabras únicas hay en nuestro pequeño corpus
# En este caso son 9 palabras diferentes

len(vectorizer.get_feature_names_out())

9

In [11]:
# =============================================================================
# VISUALIZAR LA MATRIZ DE VECTORIZACIÓN
# =============================================================================
# Convertimos la matriz dispersa (sparse matrix) a DataFrame para visualizarla mejor
# Cada fila = un documento
# Cada columna = una palabra del vocabulario
# Cada celda = número de veces que la palabra aparece en ese documento

# Ejemplo de lectura:
# - Fila 0 (documento 0): 'document' aparece 1 vez, 'first' aparece 1 vez, 'is' aparece 1 vez...
# - Fila 1 (documento 1): 'document' aparece 2 veces (¡fíjate que cuenta!)

pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

,and,document,first,is,one,second,the,third,this
0,0,1,1,1,0,0,1,0,1
1,0,2,0,1,0,1,1,0,1
2,1,0,0,1,1,0,1,1,1
3,0,1,1,1,0,0,1,0,1


In [12]:
# =============================================================================
# VECTORIZACIÓN BINARIA - Para nuestras reviews de películas
# =============================================================================
# Ahora aplicamos CountVectorizer a nuestras reviews reales con binary=True
# 
# ¿Por qué binary=True?
# - En lugar de contar cuántas veces aparece cada palabra (1, 2, 3, ...)
# - Solo marcamos si la palabra está presente (1) o ausente (0)
# - Esto suele funcionar mejor para análisis de sentimiento
# - Evita que palabras muy repetidas dominen el modelo

baseline_vectorizer = CountVectorizer(binary=True)

# fit(): el vectorizador "aprende" todas las palabras únicas de las 25,000 reviews de train
baseline_vectorizer.fit(reviews_train_clean)

# transform(): convierte cada review en un vector numérico
# ¡IMPORTANTE! Aplicamos el MISMO vectorizador a test (mismo vocabulario, mismas columnas)
X_baseline = baseline_vectorizer.transform(reviews_train_clean)
X_test_baseline = baseline_vectorizer.transform(reviews_test_clean)

In [13]:
# =============================================================================
# INSPECCIONAR LA MATRIZ RESULTANTE
# =============================================================================
# X_baseline es una matriz dispersa (sparse matrix) porque tiene muchos ceros
# Dimensiones: (25000 filas, 87063 columnas)
# - 25,000 filas = 25,000 reviews
# - 87,063 columnas = 87,063 palabras únicas encontradas en el corpus
# - 3,410,713 elementos almacenados = celdas con valor 1 (el resto son 0)
#
# ¿Por qué sparse matrix?
# - Si guardáramos todos los valores (incluyendo ceros) ocuparía muchísima memoria
# - Sparse solo guarda las posiciones con 1, ahorrando espacio

X_baseline

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 3410713 stored elements and shape (25000, 87063)>

In [14]:
# =============================================================================
# VOCABULARIO DEL VECTORIZADOR
# =============================================================================
# Mostramos el tamaño de la matriz: (25000 reviews, 87063 palabras únicas)
print(X_baseline.shape)

# El atributo vocabulary_ es un diccionario donde:
# - Clave: la palabra
# - Valor: el índice/columna que le corresponde en la matriz
# Ejemplo: {'good': 35421, 'bad': 8792, ...}
# Esto significa que la columna 35421 representa la palabra 'good'

baseline_vectorizer.vocabulary_

(25000, 87063)


{'bromwell': 9819,
 'high': 35211,
 'is': 39472,
 'cartoon': 11686,
 'comedy': 14754,
 'it': 39642,
 'ran': 61772,
 'at': 4537,
 'the': 76725,
 'same': 66138,
 'time': 77626,
 'as': 4211,
 'some': 71188,
 'other': 54861,
 'programs': 60156,
 'about': 284,
 'school': 67025,
 'life': 44297,
 'such': 74177,
 'teachers': 75997,
 'my': 51490,
 'years': 86260,
 'in': 37733,
 'teaching': 76000,
 'profession': 60088,
 'lead': 43605,
 'me': 47976,
 'to': 77922,
 'believe': 6894,
 'that': 76671,
 'satire': 66462,
 'much': 51018,
 'closer': 14125,
 'reality': 62242,
 'than': 76643,
 'scramble': 67253,
 'survive': 74812,
 'financially': 27904,
 'insightful': 38615,
 'students': 73768,
 'who': 84627,
 'can': 11132,
 'see': 67695,
 'right': 64476,
 'through': 77369,
 'their': 76762,
 'pathetic': 56362,
 'pomp': 58787,
 'pettiness': 57341,
 'of': 53843,
 'whole': 84639,
 'situation': 69959,
 'all': 2038,
 'remind': 63323,
 'schools': 67049,
 'knew': 42226,
 'and': 2762,
 'when': 84450,
 'saw': 66588,

In [15]:
# =============================================================================
# VECTORIZACIÓN CON CONTEO (no binaria)
# =============================================================================
# Ahora probamos SIN binary=True para ver la diferencia
# En este caso, la matriz contendrá el NÚMERO DE VECES que aparece cada palabra
# (no solo 0 o 1, sino 0, 1, 2, 3, 4, ...)

vectorizer_c = CountVectorizer()  # Sin binary=True
vectorizer_c.fit(reviews_train_clean)

# Esta matriz contendrá conteos reales
X_baseline_c = vectorizer_c.transform(reviews_train_clean)

In [17]:
# =============================================================================
# COMPARACIÓN: Conteo vs Binario
# =============================================================================
# Dimensiones: (25000, 87063) - ¡igual que antes!
# El número de palabras únicas (columnas) es el mismo
# Lo que cambia son los VALORES dentro de la matriz

print(X_baseline_c.shape)
print(len(vectorizer_c.get_feature_names_out()))  # Las mismas 87,063 palabras

# Si descomentamos la siguiente línea, veríamos la matriz completa
# pero es ENORME (25000 x 87063 = 2,176,575,000 celdas!)
X_baseline_c.toarray()

(25000, 87063)
87063


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(25000, 87063))

In [18]:
# =============================================================================
# REPRESENTACIÓN DE LA MATRIZ SPARSE
# =============================================================================
# Matriz demasiado grande como para que numpy la imprima completa por pantalla
# Python nos muestra el formato comprimido (Compressed Sparse Row)
# 3,410,713 elementos almacenados de 2+ mil millones de posiciones posibles

X_baseline_c

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 3410713 stored elements and shape (25000, 87063)>

# Entrenar un Modelo Base

Entrenar un modelo de Regresión Logística después de transformar los datos con CountVectorizer

* Son fáciles de interpretar
* Los modelos lineales tienden a funcionar bien en conjuntos de datos dispersos como este
* Aprenden muy rápido en comparación con otros algoritmos.

Probar modelos con valores de C de [0.01, 0.05, 0.25, 0.5, 1] y ver cuál es el mejor valor para C, y calcular la precisión

In [19]:
# =============================================================================
# MODELO BASELINE - Regresión Logística con Grid Search
# =============================================================================
# Vamos a entrenar nuestro primer modelo de clasificación para predecir el sentimiento

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# PASO 1: Crear las etiquetas (targets)
# Las reviews están ordenadas: primeras 12,500 son positivas (1), últimas 12,500 negativas (0)
target = [1 if i < 12500 else 0 for i in range(25000)]

# Equivalente más explícito (comentado):
# target = []
# for i in range(25000):
#     if i < 12500:
#         target.append(1)  # Positivo
#     else:
#         target.append(0)  # Negativo

# PASO 2: Función para entrenar el modelo con validación cruzada
def train_model(X_TRAIN, X_TEST):
    """
    Entrena un modelo de Regresión Logística con Grid Search
    
    Grid Search:
    - Prueba diferentes valores del hiperparámetro C
    - C controla la regularización (penalización por complejidad)
    - C pequeño = más regularización = modelo más simple
    - C grande = menos regularización = modelo más complejo
    
    Cross-Validation (cv=5):
    - Divide los datos de entrenamiento en 5 partes
    - Entrena 5 veces, cada vez usando 4 partes para entrenar y 1 para validar
    - Esto nos da una estimación más robusta del rendimiento
    
    Args:
        X_TRAIN: matriz de features de entrenamiento
        X_TEST: matriz de features de test
    """
    
    lr = LogisticRegression()  # Creamos el modelo
    
    # Hiperparámetros a probar
    params = {
        'C': [0.01, 0.05, 0.25, 0.5, 1]
    }
    
    # GridSearchCV prueba todas las combinaciones y elige la mejor
    grid = GridSearchCV(lr, params, cv=5)
    grid.fit(X_TRAIN, target)
    
    # Evaluamos el mejor modelo encontrado en el conjunto de test
    print("Final Accuracy: %s" % accuracy_score(target, grid.best_estimator_.predict(X_TEST)))

In [20]:
# =============================================================================
# ENTRENAR Y EVALUAR EL MODELO BASELINE
# =============================================================================
# Entrenamos el modelo con las matrices vectorizadas (binarias)
# Este es nuestro BASELINE - el modelo más simple contra el que compararemos mejoras

# X_baseline: reviews de train vectorizadas
# X_test_baseline: reviews de test vectorizadas

train_model(X_baseline, X_test_baseline)

# Resultado: ~88.18% de accuracy
# Esto significa que el modelo clasifica correctamente el 88.18% de las reviews
# ¡No está mal para un primer intento!

Final Accuracy: 0.88184


# Eliminar Stop Words

Las stop words son palabras muy comunes como 'if', 'but', 'we', 'he', 'she' y 'they'. Normalmente podemos eliminar estas palabras sin cambiar la semántica de un texto y hacerlo a menudo (pero no siempre) mejora el rendimiento de un modelo. Eliminar estas stop words se vuelve mucho más útil cuando comenzamos a usar secuencias de palabras más largas como características del modelo (ver n-gramas más adelante).

Antes de aplicar el CountVectorizer, eliminemos las stopwords incluidas en nltk.corpus

Luego aplicar el CountVectorizer, entrenar el modelo de Regresión Logística y obtener la precisión.

In [50]:
#%pip install nltk

In [21]:
# =============================================================================
# VERIFICAR VERSIÓN DE NUMPY
# =============================================================================
# Comprobamos la versión de numpy instalada
# Nota: versiones antiguas pueden tener compatibilidad diferente con otras librerías

np.__version__
# Versión antigua era 1.26.4, ahora está actualizada a 2.2.6

'2.2.6'

In [22]:
# =============================================================================
# DESCARGAR STOPWORDS DE NLTK
# =============================================================================
# NLTK (Natural Language Toolkit) es una librería muy popular para NLP
# Necesitamos descargar el dataset de stopwords la primera vez que lo usamos

import nltk
nltk.download('stopwords')  # Descarga las stopwords en múltiples idiomas

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\borja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
# =============================================================================
# VISUALIZAR STOPWORDS EN INGLÉS
# =============================================================================
# ¿Qué son las stopwords?
# Son palabras muy comunes que aparecen en casi todos los textos
# Ejemplos: 'the', 'a', 'is', 'in', 'of', 'and', etc.
# 
# ¿Por qué eliminarlas?
# - No aportan mucho significado al sentimiento
# - Reducen el ruido en el modelo
# - Disminuyen el tamaño del vocabulario (menos features)

from nltk.corpus import stopwords

# Mostramos las primeras 20 stopwords en inglés
stopwords.words('english')[:20]

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been']

In [24]:
# =============================================================================
# CONTAR STOPWORDS EN INGLÉS
# =============================================================================
# NLTK tiene 198 stopwords predefinidas para inglés

len(stopwords.words('english'))

198

In [25]:
# =============================================================================
# STOPWORDS EN OTROS IDIOMAS - Ejemplo en español
# =============================================================================
# NLTK incluye stopwords para múltiples idiomas
# Aquí vemos las primeras 20 en español

stopwords.words('spanish')[:20]

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo']

In [26]:
# =============================================================================
# CANTIDAD DE STOPWORDS EN ESPAÑOL
# =============================================================================
# El español tiene 313 stopwords en NLTK (más que inglés)
# Esto depende de la complejidad morfológica de cada idioma

len(stopwords.words('spanish'))

313

In [27]:
# =============================================================================
# STOPWORDS EN CHINO
# =============================================================================
# Ejemplo de stopwords en chino
# Los idiomas asiáticos tienen sistemas de escritura muy diferentes

stopwords.words('chinese')[:10]

['一', '一下', '一些', '一切', '一则', '一天', '一定', '一方面', '一旦', '一时']

In [28]:
# =============================================================================
# CANTIDAD DE STOPWORDS EN CHINO
# =============================================================================
# El chino tiene 841 stopwords - ¡mucho más que inglés o español!

len(stopwords.words('chinese'))

841

In [29]:
# =============================================================================
# STOPWORDS EN EUSKERA (VASCO)
# =============================================================================
# NLTK incluye hasta idiomas menos comunes como el euskera
# Primeras 10 stopwords en vasco

stopwords.words('basque')[:10]

['ahala',
 'aitzitik',
 'al',
 'ala ',
 'alabadere',
 'alabaina',
 'alabaina',
 'aldiz ',
 'alta',
 'amaitu']

In [30]:
# =============================================================================
# CANTIDAD DE STOPWORDS EN EUSKERA
# =============================================================================
# El euskera tiene 326 stopwords en NLTK

len(stopwords.words('basque'))

326

In [31]:
# =============================================================================
# MÉTODO 1: ELIMINAR STOPWORDS MANUALMENTE (antes de vectorizar)
# =============================================================================
# Este método elimina stopwords aplicando un filtro sobre las reviews
# Es el enfoque "manual" antes de usar CountVectorizer

from nltk.corpus import stopwords

# Cargamos la lista de stopwords en inglés
english_stop_words = stopwords.words('english')

def remove_stop_words(corpus):
    """
    Elimina stopwords de cada review en el corpus
    
    Proceso:
    1. Para cada review, la dividimos en palabras (split())
    2. Filtramos las palabras que NO están en la lista de stopwords
    3. Unimos las palabras que quedan de nuevo en un string
    
    Args:
        corpus: lista de reviews (strings)
    
    Returns:
        removed_stop_words: lista de reviews sin stopwords
    """
    removed_stop_words = []
    for review in corpus:
        # List comprehension que:
        # - Divide la review en palabras (review.split())
        # - Convierte cada palabra a minúsculas
        # - Solo incluye palabras que NO están en english_stop_words
        # - Une todo con espacios (' '.join())
        removed_stop_words.append(
            ' '.join([word.lower() for word in review.split() 
                      if word.lower() not in english_stop_words])
        )
    
    return removed_stop_words

# Aplicamos la eliminación de stopwords ANTES de vectorizar
no_stop_words_train = remove_stop_words(reviews_train_clean)
no_stop_words_test = remove_stop_words(reviews_test_clean)

# NOTA: Este método es más "manual" y menos eficiente que usar el parámetro
# stop_words del CountVectorizer (ver más abajo)

In [32]:
# =============================================================================
# VECTORIZAR DESPUÉS DE ELIMINAR STOPWORDS (método manual)
# =============================================================================
# Ahora vectorizamos las reviews que ya tienen las stopwords eliminadas
# 
# Documentación de CountVectorizer:
# - lowercase=True: convierte todo a minúsculas antes de vectorizar (por defecto)
# - stop_words: permite pasar una lista de stopwords (¡mejor método, ver abajo!)

cv = CountVectorizer(binary=True)
cv.fit(no_stop_words_train)

X = cv.transform(no_stop_words_train)

In [33]:
# =============================================================================
# COMPROBAR DIMENSIONES
# =============================================================================
# Verificamos el tamaño de la matriz después de eliminar stopwords
# Esperamos menos columnas (palabras) que antes

print(X.shape)  # Debería ser (25000, algo menos que 87063)

(25000, 87046)


In [34]:
# =============================================================================
# ENTRENAR MODELO CON STOPWORDS ELIMINADAS (método manual)
# =============================================================================
# Aplicamos la misma transformación a test y entrenamos el modelo

X_test = cv.transform(no_stop_words_test)

# Evaluamos: ¿mejora o empeora el accuracy?
train_model(X, X_test)

# Resultado: ~87.9% - ¡ligeramente peor que el baseline!
# Esto puede pasar: no siempre eliminar stopwords mejora el modelo

Final Accuracy: 0.879


In [35]:
# =============================================================================
# COMPARACIÓN: ¿Cuántas palabras eliminamos?
# =============================================================================
# Comparamos el número de features (columnas) antes y después

print(X_baseline.shape)  # Baseline: 87,063 palabras
print(X.shape)           # Con stopwords eliminadas: 87,046 palabras
print("Stop words eliminadas:", X_baseline.shape[1] - X.shape[1])

# ¡Solo eliminó 17 palabras! ¿Por qué tan pocas?
# Porque nuestra función remove_stop_words() hace un split() simple
# que no tokeniza tan bien como CountVectorizer
# Por ejemplo: "it's" no se separa correctamente en "it" y "'s"

(25000, 87063)
(25000, 87046)
Stop words eliminadas: 17


In [36]:
# =============================================================================
# MÉTODO 2: ELIMINAR STOPWORDS CON CountVectorizer (MEJOR MÉTODO)
# =============================================================================
# Este es el método RECOMENDADO: pasar stop_words directamente al vectorizador
# 
# Ventajas:
# - Más simple (menos código)
# - Más eficiente
# - Mejor tokenización (CountVectorizer es más inteligente)
# - Elimina más stopwords correctamente

cv = CountVectorizer(binary=True,
                     stop_words=english_stop_words)  # ¡Solo añadimos este parámetro!

# Aplicamos sobre las reviews originales (limpias, pero sin eliminar stopwords manualmente)
cv.fit(reviews_train_clean)

X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

# train_model(X, X_test)  # Comentado para no ejecutar ahora

In [37]:
# =============================================================================
# COMPARACIÓN: Método manual vs CountVectorizer
# =============================================================================
# Comparamos cuántas stopwords eliminó cada método

print(X_baseline.shape)  # Baseline: 87,063 palabras
print(X.shape)           # Con CountVectorizer stop_words: 86,918 palabras
print("Stop words eliminadas:", X_baseline.shape[1] - X.shape[1])

# ¡Ahora sí! Eliminó 145 palabras
# Mucho mejor que las 17 del método manual
# 
# ¿Por qué?
# CountVectorizer tokeniza mejor (separa "it's" en "it" y "'s")
# Luego elimina todas las stopwords correctamente

(25000, 87063)
(25000, 86918)
Stop words eliminadas: 145


**Nota:** En la práctica, una forma más fácil de eliminar stop words es simplemente usar el argumento stop_words con cualquiera de las clases 'Vectorizer' de scikit-learn. Si quieres usar la lista completa de stop words de NLTK puedes hacer stop_words='english'. En la práctica he encontrado que usar la lista de NLTK en realidad disminuye mi rendimiento porque es demasiado expansiva, así que normalmente proporciono mi propia lista de palabras. Por ejemplo, stop_words=['in','of','at','a','the'].

Un siguiente paso común en el preprocesamiento de texto es normalizar las palabras en tu corpus intentando convertir todas las diferentes formas de una palabra dada en una. Dos métodos que existen para esto son Stemming y Lemmatization.

# Stemming

Stemming se considera el enfoque más crudo/de fuerza bruta para la normalización (aunque esto no necesariamente significa que tendrá peor rendimiento). Hay varios algoritmos, pero en general todos usan reglas básicas para cortar los finales de las palabras.

NLTK tiene varias implementaciones de algoritmos de stemming. Usaremos el Porter stemmer. Los más usados:
* PorterStemmer
* SnowballStemmer

Aplicar un PorterStemmer, vectorizar y entrenar el modelo nuevamente

In [38]:
# =============================================================================
# STEMMING - Reducción de palabras a su raíz (método "bruto")
# =============================================================================
# ¿Qué es stemming?
# Es un proceso que recorta las palabras eliminando sufijos para obtener su "raíz"
# Es un método rápido pero "bruto" (no siempre linguísticamente correcto)
#
# Ejemplos:
# - "running", "runs", "ran" → "run"
# - "cats", "catty" → "cat"
# - "flies", "flying" → "fli"
#
# Algoritmos de stemming:
# - PorterStemmer: el más común, desarrollado por Martin Porter en 1980
# - SnowballStemmer: una mejora del PorterStemmer
# - LancasterStemmer: más agresivo (recorta más)

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

# Lista de palabras de ejemplo en diferentes formas
plurals = ['caresses', 'flies', 'fly','flight', 'flown', 'dies', 'die', 'mules', 'denied', 'deny',
            'died', 'agreed','agree', 'owned', 'humbled', 'sized',
            'meeting', 'stating', 'siezing', 'itemization',
            'sensational', 'traditional', 'reference', 'colonizer', 'colonizing',
            'plotted']

# Aplicamos stemming a cada palabra
singles = [stemmer.stem(plural) for plural in plurals]

# Observa cómo se recortan:
# - 'flies' y 'fly' → 'fli' (pierde sentido)
# - 'agreed' y 'agree' → 'agre'
# - 'sensational' → 'sensat'
print(' '.join(singles))

caress fli fli flight flown die die mule deni deni die agre agre own humbl size meet state siez item sensat tradit refer colon colon plot


In [39]:
# =============================================================================
# SNOWBALL STEMMER - Versión mejorada del Porter Stemmer
# =============================================================================
# SnowballStemmer es una evolución del PorterStemmer
# Ventaja: soporta múltiples idiomas (15 idiomas diferentes)
# Sintaxis: SnowballStemmer('idioma')

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')  # Especificamos el idioma

plurals = ['caresses', 'flies', 'fly','flight', 'dies', 'mules', 'denied', 'deny',
            'died', 'agreed', 'owned', 'humbled', 'sized',
            'meeting', 'stating', 'siezing', 'itemization',
            'sensational', 'traditional', 'reference', 'colonizer',
            'plotted']

singles = [stemmer.stem(plural) for plural in plurals]

# Los resultados son muy similares al PorterStemmer
# Pero Snowball suele ser ligeramente más preciso
print(' '.join(singles))

caress fli fli flight die mule deni deni die agre own humbl size meet state siez item sensat tradit refer colon plot


In [40]:
# =============================================================================
# SNOWBALL STEMMER EN ESPAÑOL
# =============================================================================
# Ejemplo de stemming en español
# Es importante porque cada idioma tiene reglas morfológicas diferentes

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('spanish')  # Cambiamos el idioma

plurals = ['recorrer', 'corriendo', 'correlación', 'correré', 
           'casas', 'casero', 'caso', 'playa', 
           'volando', 'volar', 'volveré']

singles = [stemmer.stem(plural) for plural in plurals]

# Observa los resultados:
# - 'recorrer', 'corriendo', 'correré' → 'recorr', 'corr', 'corr'
# - 'casas', 'casero', 'caso' → 'cas', 'caser', 'cas'
# - 'volando', 'volar', 'volveré' → 'vol', 'vol', 'volv'
print(' '.join(singles))

recorr corr correl corr cas caser cas play vol vol volv


In [41]:
# =============================================================================
# APLICAR STEMMING A NUESTRAS REVIEWS
# =============================================================================
# Ahora aplicamos stemming a todas las reviews de películas
# Objetivo: reducir el vocabulario agrupando palabras similares

from nltk.stem.porter import PorterStemmer

def get_stemmed_text(corpus):
    """
    Aplica stemming a cada palabra de cada review
    
    Proceso:
    1. Para cada review, dividirla en palabras
    2. Aplicar stemming a cada palabra
    3. Unir las palabras procesadas de nuevo en un string
    
    Args:
        corpus: lista de reviews
    
    Returns:
        lista de reviews con palabras "stemmed"
    """
    stemmer = PorterStemmer()
    
    # Para cada review:
    # - Dividir en palabras (.split())
    # - Aplicar stemming a cada palabra (stemmer.stem())
    # - Unir con espacios (' '.join())
    return [' '.join([stemmer.stem(word) for word in review.split()]) 
            for review in corpus]

# Aplicamos stemming a train y test
stemmed_reviews_train = get_stemmed_text(reviews_train_clean)
stemmed_reviews_test = get_stemmed_text(reviews_test_clean)

# Ahora vectorizamos las reviews "stemmed"
cv = CountVectorizer(binary=True, stop_words=english_stop_words)
cv.fit(stemmed_reviews_train)

X_stem = cv.transform(stemmed_reviews_train)
X_test = cv.transform(stemmed_reviews_test)

In [42]:
# =============================================================================
# EVALUAR MODELO CON STEMMING
# =============================================================================
# Entrenamos el modelo con las reviews procesadas con stemming
# ¿Mejora el accuracy?

train_model(X_stem, X_test)

# Resultado: ~87.68%
# Similar al modelo anterior (sin stemming con stopwords eliminadas)
# Stemming no siempre mejora, pero reduce significativamente el vocabulario

Final Accuracy: 0.8768


In [43]:
# =============================================================================
# IMPACTO DEL STEMMING EN EL VOCABULARIO
# =============================================================================
# Comparamos el tamaño del vocabulario antes y después del stemming

print(X_baseline.shape)      # Sin stemming: 87,063 palabras
print(X_stem.shape)          # Con stemming: 66,715 palabras
print("Diff X normal y X tras stemmer y vectorización:", 
      X_baseline.shape[1] - X_stem.shape[1])

# ¡Reducción de 20,348 palabras!
# Esto es una reducción del ~23% del vocabulario
# 
# ¿Por qué?
# Porque stemming agrupa formas de la misma palabra:
# - "amazing", "amazed", "amazement" → "amaz"
# - "loving", "loved", "loves" → "love"
# 
# Ventajas:
# - Menos features = modelo más rápido de entrenar
# - Agrupa conceptos similares
# - Reduce overfitting
#
# Desventajas:
# - Pierde matices del lenguaje
# - Puede crear "palabras" sin sentido ("fli" en lugar de "fly")

(25000, 87063)
(25000, 66715)
Diff X normal y X tras stemmer y vectorización: 20348


# Lemmatization

La lemmatization funciona identificando la parte del discurso de una palabra dada y luego aplicando reglas más complejas para transformar la palabra en su raíz verdadera.

In [74]:
import nltk

In [44]:
# =============================================================================
# DESCARGAR WORDNET - Base de datos léxica para lemmatization
# =============================================================================
# WordNet es una base de datos léxica del inglés
# Contiene información sobre relaciones entre palabras, sinónimos, etc.
# Es necesaria para hacer lemmatization correctamente

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\borja\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [45]:
# =============================================================================
# LEMMATIZATION - Reducción de palabras a su forma base (método "inteligente")
# =============================================================================
# ¿Qué es lemmatization?
# Es un proceso más sofisticado que stemming
# Usa un diccionario (WordNet) para encontrar la forma base real de cada palabra
#
# Diferencias clave con stemming:
# - Stemming: recorta sufijos (método bruto) → "flies" → "fli"
# - Lemmatization: busca la raíz real → "flies" → "fly"
#
# Ventajas de lemmatization:
# - Produce palabras reales (no "raíces inventadas")
# - Más preciso linguísticamente
# - Considera el contexto (parte del discurso: verbo, sustantivo, etc.)
#
# Desventajas:
# - Más lento que stemming (requiere búsqueda en diccionario)
# - Necesita WordNet u otro diccionario
# - No disponible fácilmente en todos los idiomas

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Mismas palabras de ejemplo que con stemming
plurals = ['caresses', 'flies','fly','flight', 'dies', 'mules', 'studies',
            'died', 'agreed', 'owned', 'humbled', 'sized',
            'meeting', 'stating', 'siezing', 'itemization',
            'sensational', 'traditional', 'reference', 'colonizer',
            'plotted']

# Aplicamos lemmatization
singles = [lemmatizer.lemmatize(plural) for plural in plurals]

# Compara con stemming:
# - Stemming: 'flies' → 'fli' (sin sentido)
# - Lemmatization: 'flies' → 'fly' (palabra real)
#
# - Stemming: 'agreed' → 'agre'
# - Lemmatization: 'agreed' → 'agreed' (no detecta que es verbo sin contexto)
#
# Nota: por defecto, lemmatize() asume que la palabra es un sustantivo
# Para mejor precisión se puede especificar: lemmatize(word, pos='v') para verbos
print(' '.join(singles))

caress fly fly flight dy mule study died agreed owned humbled sized meeting stating siezing itemization sensational traditional reference colonizer plotted


In [46]:
# =============================================================================
# APLICAR LEMMATIZATION A NUESTRAS REVIEWS
# =============================================================================
# Aplicamos lemmatization a todas las reviews de películas
# El proceso es similar a stemming pero con resultados más precisos

def get_lemmatized_text(corpus):
    """
    Aplica lemmatization a cada palabra de cada review
    
    Proceso similar a get_stemmed_text() pero usando WordNetLemmatizer
    
    Args:
        corpus: lista de reviews
    
    Returns:
        lista de reviews con palabras lemmatizadas
    """
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Para cada review:
    # - Dividir en palabras
    # - Aplicar lemmatization a cada palabra
    # - Unir de nuevo
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) 
            for review in corpus]

# Lemmatizamos las reviews
lemmatized_reviews_train = get_lemmatized_text(reviews_train_clean)
lemmatized_reviews_test = get_lemmatized_text(reviews_test_clean)

# Vectorizamos con conteo tras lematizar
cv = CountVectorizer(binary=True, stop_words=english_stop_words)
cv.fit(lemmatized_reviews_train)

X = cv.transform(lemmatized_reviews_train)
X_test = cv.transform(lemmatized_reviews_test)

# Entrenamos y evaluamos
train_model(X, X_test)

# Resultado: ~87.82%
# Similar a stemming, ligeramente mejor

Final Accuracy: 0.87824


In [47]:
# =============================================================================
# IMPACTO DE LEMMATIZATION EN EL VOCABULARIO
# =============================================================================
# Comparamos el tamaño del vocabulario con lemmatization vs baseline

print(X_baseline.shape)  # Baseline: 87,063 palabras
print(X.shape)           # Con lemmatization: 80,215 palabras
print("Diff X normal y X tras lematizador y vectorización:", 
      X_baseline.shape[1] - X.shape[1])

# Reducción de 6,848 palabras (7.9%)
# Mucho MENOS reducción que stemming (que eliminó 20,348 palabras)
#
# ¿Por qué?
# Lemmatization es menos agresivo:
# - Stemming: "running", "runs", "ran" → "run" (todas iguales)
# - Lemmatization: "running" → "running", "runs" → "run", "ran" → "ran"
#   (preserva algunas diferencias si no tiene contexto)
#
# Conclusión:
# - Lemmatization: más preciso, preserva más información
# - Stemming: más agresivo, reduce más el vocabulario
# - En este caso, ninguno mejora significativamente el accuracy del baseline

(25000, 87063)
(25000, 80215)
Diff X normal y X tras lematizador y vectorización: 6848


# n-gramas

Potencialmente podemos agregar más poder predictivo a nuestro modelo agregando secuencias de dos o tres palabras (bigramas o trigramas) también. Por ejemplo, si una reseña tiene la secuencia de tres palabras "didn't love movie" solo consideraríamos estas palabras individualmente con un modelo de solo unigramas y probablemente no capturaríamos que esto es en realidad un sentimiento negativo porque la palabra 'love' por sí misma estará altamente correlacionada con una reseña positiva.

La librería scikit-learn hace esto muy fácil de probar. Solo usa el argumento ngram_range con cualquiera de las clases 'Vectorizer'.

In [48]:
# =============================================================================
# N-GRAMAS - Capturando secuencias de palabras
# =============================================================================
# ¿Qué son los n-gramas?
# Son secuencias de N palabras consecutivas en un texto
#
# Tipos:
# - Unigrama (n=1): palabras individuales → "didn't", "love", "music"
# - Bigrama (n=2): pares de palabras → "didn't love", "love music"
# - Trigrama (n=3): tríos de palabras → "didn't love music", "love music at"
#
# ¿Por qué usar n-gramas?
# - Capturan contexto: "not good" tiene significado negativo
# - Solo con unigramas: "not" (negativo?) + "good" (positivo?) = confuso
# - Con bigramas: "not good" es claramente negativo
#
# Ejemplos útiles en análisis de sentimiento:
# - "didn't love" (negativo, aunque "love" solo sea positivo)
# - "not bad" (positivo, aunque "not" y "bad" sean negativos)
# - "very good" (muy positivo)

from nltk import ngrams

sentence = "didn't love music at all my love"

# Creamos unigramas, bigramas y trigramas
one = ngrams(sentence.split(), 1)    # Palabras individuales
two = ngrams(sentence.split(), 2)    # Pares de palabras
three = ngrams(sentence.split(), 3)  # Tríos de palabras

# Unigramas
print("UNIGRAMAS (palabras individuales):")
for grams in one:
    print(grams)
print('###############')

# Bigramas
print("BIGRAMAS (pares de palabras):")
for grams in two:
    print(grams)
print('###############')

# Trigramas
print("TRIGRAMAS (tríos de palabras):")
for grams in three:
    print(grams)

# Observa cómo los bigramas capturan mejor el contexto:
# - ("didn't", "love") transmite negación del amor
# - Solo "love" podría parecer positivo

UNIGRAMAS (palabras individuales):
("didn't",)
('love',)
('music',)
('at',)
('all',)
('my',)
('love',)
###############
BIGRAMAS (pares de palabras):
("didn't", 'love')
('love', 'music')
('music', 'at')
('at', 'all')
('all', 'my')
('my', 'love')
###############
TRIGRAMAS (tríos de palabras):
("didn't", 'love', 'music')
('love', 'music', 'at')
('music', 'at', 'all')
('at', 'all', 'my')
('all', 'my', 'love')


In [49]:
# =============================================================================
# N-GRAMAS CON CountVectorizer - Ejemplo simple
# =============================================================================
# CountVectorizer puede generar n-gramas automáticamente con ngram_range
# 
# ngram_range es una tupla (min_n, max_n):
# - (1, 1): solo unigramas
# - (2, 2): solo bigramas
# - (1, 2): unigramas Y bigramas
# - (1, 3): unigramas, bigramas Y trigramas

ngram_vectorizer = CountVectorizer(binary=True,
                                   ngram_range=(1, 2))  # Unigramas + Bigramas

# Aplicamos al ejemplo simple
vector = ngram_vectorizer.fit_transform([sentence]).toarray()
print(vector)
print(len(vector[0]))

# El vector tiene 12 elementos:
# - 7 unigramas únicos
# - 5 bigramas únicos (6 bigramas totales, pero "love" se repite)
# 
# NOTA: algunas palabras pueden ser eliminadas automáticamente por CountVectorizer
# (por ejemplo, palabras de una sola letra como 'a')

[[1 1 1 1 1 1 1 1 1 1 1 1]]
12


In [50]:
# =============================================================================
# APLICAR N-GRAMAS A NUESTRAS REVIEWS - Con límite de features
# =============================================================================
# Ahora aplicamos n-gramas a todas nuestras reviews de películas
# 
# IMPORTANTE: ngram_range=(1, 2) crea MUCHAS features
# - Unigramas: ~87,000
# - Bigramas: cientos de miles o millones
# - Total: puede explotar en tamaño
#
# Solución: max_features limita el número de features
# Selecciona solo las N palabras/n-gramas más frecuentes

ngram_vectorizer = CountVectorizer(binary=True, 
                                   stop_words=english_stop_words,
                                   ngram_range=(1, 2),      # Unigramas + bigramas
                                   max_features=30000)      # Solo las 30,000 más frecuentes

ngram_vectorizer.fit(reviews_train_clean)

X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

# Entrenamos el modelo
train_model(X, X_test)

# Resultado: ~88.74% - ¡MEJORA respecto al baseline!
# Los bigramas ayudan al modelo a entender mejor el contexto

Final Accuracy: 0.8874


In [51]:
# =============================================================================
# IMPACTO DE max_features
# =============================================================================
# Comparamos el tamaño de features con y sin límite

print(X_baseline.shape)  # Baseline: 87,063 palabras (solo unigramas)
print(X.shape)           # Con n-gramas: 30,000 features (limitado por max_features)
print("Diff X normal y X tras n-gramas:", X_baseline.shape[1] - X.shape[1])

# Sin max_features, ¡tendríamos 1,448,047 features!
# 
# ¿Por qué tantas?
# Número de bigramas posibles = vocabulario × vocabulario
# Si tenemos ~87,000 palabras, hay millones de combinaciones posibles
#
# max_features=30000 nos da un buen balance:
# - Suficientes features para capturar patrones importantes
# - No tantas que el modelo sea inmanejable
# - Selecciona las más frecuentes/informativas

(25000, 87063)
(25000, 30000)
Diff X normal y X tras n-gramas: 57063


In [52]:
# =============================================================================
# EXPERIMENTO: Solo bigramas (sin unigramas)
# =============================================================================
# ¿Qué pasa si usamos SOLO bigramas?
# ngram_range=(2, 2) → solo pares de palabras, no palabras individuales

ngram_vectorizer = CountVectorizer(binary=True, 
                                   stop_words=english_stop_words,
                                   ngram_range=(2, 2),      # SOLO bigramas
                                   max_features=10000)      # 10,000 bigramas más frecuentes

ngram_vectorizer.fit(reviews_train_clean)

X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

print(X_baseline.shape)  # Baseline: 87,063 unigramas
print(X.shape)           # Solo bigramas: 10,000 features
print("Diff X normal y X tras n-gramas:", X_baseline.shape[1] - X.shape[1])

# Entrenamos el modelo
train_model(X, X_test)

# Resultado: ~81.54% - PEOR que el baseline
# 
# Conclusión:
# - Los bigramas solos NO son suficientes
# - Necesitamos COMBINAR unigramas + bigramas para mejor rendimiento
# - Los unigramas capturan palabras individuales importantes
# - Los bigramas capturan contexto y relaciones entre palabras

(25000, 87063)
(25000, 10000)
Diff X normal y X tras n-gramas: 77063
Final Accuracy: 0.81536


# TF-IDF

Otra forma común de representar cada documento en un corpus es usar la estadística tf-idf (term frequency-inverse document frequency) para cada palabra, que es un factor de ponderación que podemos usar en lugar de representaciones binarias o de conteo de palabras.

Hay varias formas de hacer la transformación tf-idf pero en resumen, **tf-idf busca representar el número de veces que una palabra dada aparece en un documento (una reseña de película en nuestro caso) en relación con el número de documentos en el corpus en los que aparece la palabra**.

**Nota:** Ahora que hemos visto los n-gramas, cuando me refiero a 'palabras' realmente quiero decir cualquier n-grama (secuencia de palabras) si el modelo está usando una n mayor que uno.

In [53]:
# =============================================================================
# CÁLCULO MANUAL DE IDF (Inverse Document Frequency)
# =============================================================================
# Antes de usar TfidfVectorizer, entendamos cómo funciona el cálculo IDF
# 
# IDF (Inverse Document Frequency):
# Penaliza palabras muy comunes y realza palabras raras
# Fórmula: 1 + ln((N + 1) / (count + 1))
# 
# Donde:
# - N = número total de documentos en el corpus
# - count = número de documentos que contienen la palabra
#
# Ejemplo: si una palabra aparece en muchos documentos, su IDF será bajo

# Número de documentos
N = 3

# Número de veces que aparece una palabra específica (en cuántos documentos)
count = 2

# Calculamos el IDF
# Si la palabra aparece en 2 de 3 documentos:
# IDF = 1 + ln((3 + 1) / (2 + 1)) = 1 + ln(4/3) = 1.288
1 + np.log((N + 1)/(count + 1))

np.float64(1.2876820724517808)

In [54]:
# =============================================================================
# TF-IDF - Term Frequency - Inverse Document Frequency
# =============================================================================
# TF-IDF combina dos métricas:
# 
# 1. TF (Term Frequency): 
#    ¿Cuántas veces aparece la palabra en un documento específico?
# 
# 2. IDF (Inverse Document Frequency):
#    ¿Qué tan rara/común es la palabra en todo el corpus?
#    - Palabras muy comunes (ej: "the", "is") → IDF bajo
#    - Palabras raras pero informativas → IDF alto
#
# TF-IDF = TF × IDF
#
# ¿Por qué usar TF-IDF en lugar de conteos simples?
# - Reduce el peso de palabras muy comunes
# - Aumenta el peso de palabras distintivas
# - Mejor representación del "contenido" real del documento
#
# Ejemplo práctico:
# - Palabra "ralph" aparece en 3 documentos → IDF = 1.0 (muy común en este corpus)
# - Palabra "nice" aparece en 1 documento → IDF = 1.69 (más distintiva)

from sklearn.feature_extraction.text import TfidfVectorizer

# Corpus de ejemplo
sent1 = 'My name is Ralph'
sent2 = 'Ralph is nice'
sent3 = 'Ralph'

# Creamos el TfidfVectorizer
test = TfidfVectorizer()
test.fit_transform([sent1, sent2, sent3])

# Valores IDF para cada palabra
# Cuanto más común es la palabra, más bajo es su IDF
print(test.idf_)
print(test.get_feature_names_out())

# Resultados:
# 'ralph': 1.0 (aparece en los 3 documentos, muy común)
# 'my', 'name', 'nice': 1.69 (aparecen en 1 solo documento, distintivas)
# 'is': 1.29 (aparece en 2 documentos, medianamente común)

[1.28768207 1.69314718 1.69314718 1.69314718 1.        ]
['is' 'my' 'name' 'nice' 'ralph']


In [55]:
# =============================================================================
# VERIFICAR EL CÁLCULO IDF MANUALMENTE
# =============================================================================
# Verificamos que el cálculo coincide con lo que hace TfidfVectorizer

# 'ralph' aparece en 3 documentos de un total de 3
# Fórmula: 1 + ln((N + 1) / (count + 1))
# 1 + ln((3 + 1) / (3 + 1)) = 1 + ln(4/4) = 1 + ln(1) = 1 + 0 = 1.0

1 + np.log((3 + 1)/(3 + 1))

np.float64(1.0)

In [56]:
# =============================================================================
# APLICAR TF-IDF A NUESTRAS REVIEWS
# =============================================================================
# Ahora aplicamos TfidfVectorizer en lugar de CountVectorizer
# Esto convierte cada review en un vector de valores TF-IDF
# en lugar de simples conteos binarios o frecuencias

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Creamos el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Ajustamos y transformamos las reviews
tfidf_vectorizer.fit(reviews_train_clean)
X = tfidf_vectorizer.transform(reviews_train_clean)
print(X.shape)

# Transformamos test con el mismo vectorizador
X_test = tfidf_vectorizer.transform(reviews_test_clean)

# Ahora cada celda de la matriz NO contiene 0 o 1
# Contiene valores continuos (floats) que representan TF-IDF
# Valores más altos = palabras más importantes para ese documento específico

(25000, 87063)


In [57]:
# =============================================================================
# ENTRENAR MODELO CON TF-IDF
# =============================================================================
# Entrenamos Regresión Logística con las features TF-IDF
# ¿Mejora respecto al baseline con CountVectorizer?

train_model(X, X_test)

# Resultado: ~88.18% 
# Prácticamente IGUAL que el baseline con CountVectorizer binario
# 
# Conclusión:
# - TF-IDF no siempre es mejor que conteos binarios
# - Para análisis de sentimiento, la presencia/ausencia (binario) suele funcionar bien
# - TF-IDF es más útil en tareas como búsqueda de información o clasificación de tópicos

Final Accuracy: 0.88176


In [58]:
# =============================================================================
# COMPARACIÓN DE DIMENSIONES: TF-IDF vs Baseline
# =============================================================================
# Comparamos el tamaño de las matrices

print(X_baseline.shape)  # CountVectorizer binario: 87,063 palabras
print(X.shape)           # TfidfVectorizer: 87,063 palabras
print("Diff X normal y X tras TF-IDF:", X_baseline.shape[1] - X.shape[1])

# ¡Mismo número de features! (diferencia = 0)
# Lo que cambia son los VALORES dentro de la matriz:
# - CountVectorizer binario: solo 0 y 1
# - TfidfVectorizer: valores continuos (floats) entre 0 y ~1

(25000, 87063)
(25000, 87063)
Diff X normal y X tras TF-IDF: 0


# Máquinas de Vectores de Soporte (SVM)

Recordemos que los clasificadores lineales tienden a funcionar bien en conjuntos de datos muy dispersos (como el que tenemos). Otro algoritmo que puede producir excelentes resultados con un tiempo de entrenamiento rápido son las Máquinas de Vectores de Soporte con un kernel lineal.

Construir un modelo con un rango de n-gramas de 1 a 2:

In [59]:
# =============================================================================
# SVM (Support Vector Machine) CON N-GRAMAS
# =============================================================================
# Ahora probamos un algoritmo diferente: SVM con kernel lineal
# 
# ¿Por qué SVM?
# - Funciona muy bien con datos de alta dimensionalidad (muchas features)
# - Especialmente bueno con datos sparse (matrices con muchos ceros)
# - A menudo más rápido que Regresión Logística en estos casos
# - Encuentra el hiperplano óptimo que separa las clases
#
# LinearSVC = Linear Support Vector Classifier
# "Linear" porque usa un kernel lineal (decisiones lineales)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Configuramos CountVectorizer con n-gramas (1-3)
# Incluimos unigramas, bigramas Y trigramas
ngram_vectorizer = CountVectorizer(binary=True, 
                                   ngram_range=(1, 3),      # Hasta trigramas
                                   max_features=20000)      # Limitamos a 20,000 features

ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

# Función para entrenar SVM con Grid Search
def train_model_svm(X_TRAIN, X_TEST):
    """
    Entrena un modelo SVM con Grid Search
    
    Hiperparámetro C:
    - Similar a Regresión Logística
    - C alto: modelo más complejo, puede hacer overfitting
    - C bajo: modelo más simple, puede hacer underfitting
    
    Args:
        X_TRAIN: matriz de features de entrenamiento
        X_TEST: matriz de features de test
    """
    
    svm = LinearSVC()  # Creamos el clasificador SVM
    
    # Valores de C a probar
    params = {
        'C': [0.01, 0.05, 0.25, 0.5, 1]
    }
    
    # Grid Search con validación cruzada
    grid = GridSearchCV(svm, params, cv=5)
    grid.fit(X_TRAIN, target)
    
    # Evaluamos en test
    print("Final Accuracy: %s" % accuracy_score(target, grid.best_estimator_.predict(X_TEST)))

# Entrenamos el SVM
train_model_svm(X, X_test)

# Resultado: ~88.65%
# ¡Mejor que el baseline! Los trigramas + SVM ayudan un poco más

Final Accuracy: 0.88648


# Modelo Final

Eliminar un pequeño conjunto de stop words junto con un rango de n-gramas de 1 a 3 y un clasificador de vectores de soporte lineal muestra los mejores resultados.

In [ ]:
# =============================================================================
# MODELO FINAL - Mejor configuración encontrada
# =============================================================================
# Después de probar diferentes técnicas, el mejor modelo combina:
# - N-gramas (1-3): unigramas, bigramas y trigramas
# - Lista pequeña de stopwords (no la lista completa de NLTK)
# - Regresión Logística (funciona casi tan bien como SVM)
#
# ¿Por qué esta configuración?
# - Los trigramas capturan frases completas como "not very good"
# - Eliminar TODAS las stopwords puede ser contraproducente
# - Solo eliminamos las palabras más comunes que claramente no aportan

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

# Lista REDUCIDA de stopwords
# Solo incluimos las más comunes y menos informativas
stop_words = ['in', 'of', 'at', 'a', 'the']

# CountVectorizer con la mejor configuración
ngram_vectorizer = CountVectorizer(binary=True,
                                   ngram_range=(1, 3),         # Hasta trigramas
                                   stop_words=stop_words)      # Pocas stopwords

# Ajustamos y transformamos
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

# Entrenamos con Regresión Logística
train_model(X, X_test)

# Resultado: ~90.01% - ¡EL MEJOR HASTA AHORA!
# 
# Mejoras respecto al baseline (88.18%):
# - +1.83% de accuracy
# - Mejor captura de contexto con n-gramas
# - Balance entre complejidad y generalización

# Características Positivas y Negativas Principales

Obtener las características más importantes del modelo.

In [ ]:
# =============================================================================
# ANÁLISIS DE FEATURES - Palabras más importantes
# =============================================================================
# Ahora vamos a investigar qué palabras son más importantes para el modelo
# Esto nos ayuda a entender QUÉ está aprendiendo el modelo
#
# Entrenamos un modelo simple (sin n-gramas) para análisis

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)

# Entrenamos Regresión Logística
log_reg = LogisticRegression(C=0.5)
log_reg.fit(X, target)

# Importancia de los coeficientes
# log_reg.coef_ contiene los pesos aprendidos por el modelo
# - Coeficiente positivo alto → palabra fuertemente asociada con clase positiva
# - Coeficiente negativo alto → palabra fuertemente asociada con clase negativa
print(len(log_reg.coef_[0]))  # Total: 87,063 coeficientes (uno por palabra)

# Cada palabra tiene su coeficiente asociado
cv.get_feature_names_out()

# Creamos un diccionario: palabra → coeficiente
# Esto nos permite ordenar y encontrar las palabras más importantes
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names_out(), log_reg.coef_[0]
    )
}

In [ ]:
# =============================================================================
# PRUEBA DEL MODELO - Review negativa
# =============================================================================
# Probamos el modelo con una review claramente negativa
# predict() devuelve 0 (negativo) o 1 (positivo)

log_reg.predict(cv.transform(['This movie is horrible']))

# Resultado esperado: [0] (negativo)
# ¡El modelo identifica correctamente el sentimiento negativo!

In [ ]:
# =============================================================================
# PRUEBA DEL MODELO - Review positiva
# =============================================================================
# Probamos con una review claramente positiva

log_reg.predict(cv.transform(['This movie is incredible']))

# Resultado esperado: [1] (positivo)
# ¡El modelo identifica correctamente el sentimiento positivo!

In [ ]:
# =============================================================================
# TOP 5 PALABRAS MÁS POSITIVAS Y NEGATIVAS
# =============================================================================
# Identificamos las palabras con mayor peso en cada sentimiento
# Esto nos ayuda a interpretar qué aprendió el modelo

# TOP 5 PALABRAS MÁS POSITIVAS
# Ordenamos por coeficiente de mayor a menor (reverse=True)
print("TOP 5 PALABRAS MÁS POSITIVAS:")
print("=" * 50)
for best_positive in sorted(
    feature_to_coef.items(),
    key=lambda x: x[1],      # Ordenar por coeficiente
    reverse=True)[:5]:        # Top 5
    print(f"{best_positive[0]}: {best_positive[1]:.4f}")

print('\n' + '=' * 50)
print("TOP 5 PALABRAS MÁS NEGATIVAS:")
print("=" * 50)

# TOP 5 PALABRAS MÁS NEGATIVAS
# Ordenamos por coeficiente de menor a mayor (sin reverse)
for best_negative in sorted(
    feature_to_coef.items(),
    key=lambda x: x[1])[:5]:  # Top 5 más negativas
    print(f"{best_negative[0]}: {best_negative[1]:.4f}")

# Resultados típicos:
# POSITIVAS: 'excellent', 'perfect', 'superb', 'wonderful', 'brilliant'
# NEGATIVAS: 'worst', 'waste', 'awful', 'boring', 'terrible'
#
# ¡Tiene sentido! El modelo aprendió palabras que realmente indican sentimiento
#
# Interpretación de coeficientes:
# - 'excellent': +1.38 → fuerte indicador de review positiva
# - 'worst': -2.08 → fuerte indicador de review negativa
# - Cuanto mayor el valor absoluto, más importante es la palabra para clasificar